In [2]:
import sys
import locale
from pprint import pprint
from random import uniform
from time import sleep
import datetime as dt
import requests
from bs4 import BeautifulSoup
from gestione_input import leggi_numero

if sys.platform in ['win32', 'cygwin']:
    locale.setlocale(locale.LC_ALL, 'it_IT')
else:
    locale.setlocale(locale.LC_ALL, 'it_IT.UTF-8')


BASE_URL = 'https://www.subito.it/annunci-italia/vendita/usato/'

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
}


search = input('Inserisci le parole chiave che vuoi cercare: ')

max_page = leggi_numero('Quante pagine vuoi analizzare? '
    'Premi semplicemente Invio per cercare su tutte le pagine disponibili.',
    'intero')


def scrape_page(page):
    query_str = {
        'q': search,
        'o': page
    }

    result = requests.get(BASE_URL, params=query_str, headers=HEADERS)

    if result.status_code == 200:
        soup = BeautifulSoup(result.content, 'html.parser')

        # Controlla se non è una pagina vuota
        no_results = soup.select('div[class*=ErrorLayout_container__]')
        if no_results:
            if page == 1:
                err_msg = f'La ricerca non ha prodotto risultati.'
            else:
                err_msg = f'Si è raggiunta la pagina n. {page} e qui non ci sono più risultati.'
            raise requests.RequestException(err_msg)
        
        products = soup.select('div.items__item.item-card')

        items = []

        for adv in products:
            data = {
                'name': adv.select('h2[class*=ItemTitle-module_item-title__]'),
                # 'name': adv.select('h2[class*=ItemTitle-module_item-title__]')[0] \
                #     .get_text() if adv.select('h2[class*=ItemTitle-module_item-title__]') else None,
                'price': adv.find_all('p', {'class': 'price'}),
                # 'price': adv.select('p.price'),
                'town': adv.select('span[class*=index-module_town__]'),
                'city': adv.select('span.city'),
                'shipping': adv.select('span.shipping-badge'),
                'sold': adv.select('span.item-sold-badge'),
                'date': adv.select('span[class*=index-module_date__]')
                # 'url': adv.select('a.link'),
            }

            for key, value in data.items():
                if value:
                    data[key] = value[0].get_text()
                else:
                    data[key] = None

                # data[key] = value[0].get_text() if value else None

            if data['date']:
                day, time = (data['date'].split(' alle '))
                
                if day.startswith('Oggi'):
                    data['date'] = dt.date.today()
                elif day.startswith('Ieri'):
                    data['date'] = dt.date.today() + dt.timedelta(days=1)
                else:
                    date_adv = dt.datetime.strptime(day, '%d %b')
                    this_year = dt.date.today().year
                    if dt.date.today().month > date_adv.month or \
                            (dt.date.today().day > date_adv.day and \
                            dt.date.today().month > date_adv.month):

                        data['date'] = date_adv.replace(year=this_year)
                    else:
                        data['date'] = date_adv.replace(year=this_year - 1)
            
            data.update({
                'price': data['price']
                        .replace('\xa0€', '')
                        .replace('Spedizione disponibile', '')
                        .replace('Venduto', ''),  # ) if data['price'] else None,
                'url': adv.select('a.link')[0]['href'],
            })
            
            items.append(data)
        
    return items

curr_page = 1
all_data = []

while True:
    # Se la max_page non è impostata oppure non si è ancora raggiunta la max_page
    if not max_page or curr_page <= max_page:
        try:
            items = scrape_page(curr_page)
        except requests.RequestException as err:
            print(err)
            break
        
        all_data += items
        curr_page += 1
        sleep(uniform(1, 4))
    else:
        break
    
print('N. pagine analizzate:', curr_page - 1)
pprint(all_data)

N. pagine analizzate: 1
[{'city': '(PA)',
  'date': None,
  'name': 'Pc gaming assemblato fisso completo',
  'price': '1.500',
  'shipping': None,
  'sold': None,
  'town': 'Palermo ',
  'url': 'https://www.subito.it/informatica/pc-gaming-assemblato-fisso-completo-palermo-493990627.htm'},
 {'city': '(BO)',
  'date': datetime.date(2023, 6, 1),
  'name': 'Ten-tec orion2 con filtri + Imballo',
  'price': '1.990',
  'shipping': None,
  'sold': None,
  'town': 'Bologna ',
  'url': 'https://www.subito.it/audio-video/ten-tec-orion2-con-filtri-imballo-bologna-495444488.htm'},
 {'city': '(CH)',
  'date': datetime.date(2023, 6, 2),
  'name': 'Westone spectrum',
  'price': '150',
  'shipping': None,
  'sold': None,
  'town': 'Chieti ',
  'url': 'https://www.subito.it/strumenti-musicali/westone-spectrum-chieti-495353506.htm'},
 {'city': '(AV)',
  'date': datetime.date(2023, 6, 2),
  'name': 'Mouse Logitech G502 Proteus Spectrum',
  'price': '15',
  'shipping': 'Spedizione disponibile',
  'sold': N

In [4]:
import sys
import locale
from pprint import pprint
from random import uniform
from time import sleep
import datetime as dt
import requests
from bs4 import BeautifulSoup
from gestione_input import leggi_numero

print(sys.platform)

if sys.platform in ['win32', 'cygwin']:
    locale.setlocale(locale.LC_ALL, 'Italian_Italy.1252')
else:
    locale.setlocale(locale.LC_ALL, 'it_IT')

# url_list = [
#     'https://www.subito.it/annunci-italia/vendita/usato/?q=spectrum+sinclair&o=1',
#     'https://www.subito.it/annunci-italia/vendita/usato/?q=spectrum+sinclair&o=2'
# ]

BASE_URL = 'https://www.subito.it/annunci-italia/vendita/usato/'

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
}

search = input('Inserisci le parole chiave che vuoi cercare: ')

def scrape_page(page):
    query_str = {
        'q': search,
        'o': page
    }

    result = requests.get(base_url, params=query_str, headers=fake_headers)

    if result.status_code == 200:
        soup = BeautifulSoup(result.content, 'html.parser')

        # Controllare se non è una pagina vuota
        no_result = soup.select('div[class*=ErrorLayout_container__]')
        if no_result:
            raise requests.RequestException('Non ci sono più pagine da analizzare.')

        products = soup.select('div.items__item.item-card')

        items = []

        for adv in products:
            data = {
                'name': adv.select('h2[class*=ItemTitle-module_item-title__]'),
                # 'name': adv.select('h2[class*=ItemTitle-module_item-title__]')[0] \
                #     .get_text() if adv.select('h2[class*=ItemTitle-module_item-title__]')
                'price': adv.find_all('p', {'class': 'price'}),
                'town': adv.select('span[class*=index-module_town__]'),
                'city': adv.select('span.city'),
                # 'price': adv.select('p.price'),        
                'shipping': adv.select('span.shipping-badge'),
                'sold': adv.select('span.item-sold-badge'),
                'date': adv.select('span[class*=index-module_date__]')
                # 'url': adv.select('a.link'),
            }

            for key, value in data.items():
                if value:
                    data[key] = value[0].get_text()
                else:
                    data[key] = None

                # data[key] = value[0].get_text() if value else None

            if data['date']:
                day, time = (data['date'].split(' alle '))
                
                if day.startswith('Oggi'):
                    data['date'] = date.today()
                elif day.startswith('Ieri'):
                    data['date'] = date.today() + timedelta(days=1)
                else: 
                    ...
                

            data.update({
                'price': data['price']
                        .replace('Spedizione disponibile','')
                        .replace('\xa0€', '')
                        .replace('Venduto', ''),
                'url': adv.select('a.link')[0]['href'],
            })

            # [0].get_text().replace('Spedizione disponibile', '').replace('Venduto', ''),

            items.append(data)

    return items


curr_page = 1
all_data = []

max_page = leggi_float('Quante pagine vuoi analizzare? '
    'Premi semplicemente invio per cercare su tutte le pagine disponibili.',
    None, None)

while True:
    if not max_page  or curr_page <= max_page:
    # if max_page is None or curr_page <= max_page:
        try:
            items = scrape_page(curr_page)
        except requests.RequestException as err:
            print(err)
            break

        all_data += items
        curr_page += 1
        sleep(uniform(1, 4))
    else:
        break

print('Raggiunta pagina:', curr_page)
pprint(all_data)

Raggiunta pagina: 2
[{'city': '(PA)',
  'date': None,
  'name': 'Pc gaming assemblato fisso completo',
  'price': '1.500',
  'shipping': None,
  'sold': None,
  'town': 'Palermo ',
  'url': 'https://www.subito.it/informatica/pc-gaming-assemblato-fisso-completo-palermo-493990627.htm'},
 {'city': '(VI)',
  'date': 'Oggi alle 14:18',
  'name': 'Libro “Grammar spectrum Gold”',
  'price': '15',
  'shipping': 'Spedizione disponibile',
  'sold': None,
  'town': 'Piovene Rocchette ',
  'url': 'https://www.subito.it/libri-riviste/libro-grammar-spectrum-gold-vicenza-495278510.htm'},
 {'city': '(AT)',
  'date': 'Oggi alle 14:02',
  'name': 'Anbernic RG353M Viola+custodia+128gb',
  'price': '149',
  'shipping': 'Spedizione disponibile',
  'sold': None,
  'town': "Villafranca d'Asti ",
  'url': 'https://www.subito.it/videogiochi/anbernic-rg353m-viola-custodia-128gb-asti-495276395.htm'},
 {'city': '(GE)',
  'date': 'Oggi alle 13:58',
  'name': 'Trasformatore alimentatore modellismo F.S',
  'price': 

In [6]:
from datetime import datetime, date
import locale

print(datetime.now().date())
print(datetime.today().date())

print(date.today())

# dt = datetime.strptime(' 1 Luglio 2023', "%d %B %Y")    # datetime object generation
# print(type(dt))

2023-05-31
2023-05-31
2023-05-31
